In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import json

from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

device = 'cpu'

# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
# embed_model_id = 'bkai-foundation-models/vietnamese-bi-encoder'
embed_model_id = "all-MiniLM-L6-v2"

cache_dir = "../cache/"

DB_SAVE_NAME = f"ensubsec_{embed_model_id.split('/')[-1].replace('.','-')}"
DOCUMENT_DIR = "../datasets/KALAPA_ByteBattles_2023_MEDICAL_Set1/subsections/"


In [ ]:
embed_model = SentenceTransformerEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'batch_size': 32, "normalize_embeddings": True, "device": device },
    cache_folder=cache_dir
)

loaded_db = FAISS.load_local(
    DB_SAVE_NAME,
    embeddings=embed_model
)

In [ ]:
query = """
Đâu là triệu chứng của bệnh van tim?
A. Khó thở
B. Tăng cân nhanh chóng
C. Vàng da
D. Rụng tóc
"""

result = loaded_db.similarity_search(query=query, k=1)
result

In [ ]:
context = "\n".join([r.page_content for r in result])
print(context)

In [ ]:
from llama_cpp import Llama

llm = Llama(
    # "../models/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    "../models/llama-2-7b.Q4_K_M.gguf",
    n_gpu_layers=24,
    n_ctx=8192,
    n_batch=256,
)

In [ ]:
llm(f"### Context:\n{context[:len(context) // 2]}\n\n### Question: {query}\n### Answer: ")

In [ ]:
query = """Tháng 8 vừa qua,An và Bình đi khám sức khỏe. An được chẩn đoán cận thị độ 3, Bình được chẩn đoán gan nhiễm mỡ. Làm thế nào để Bình hạn chế và giảm bớt bệnh của mình?
A. Tăng cường tiêu thụ cồn
B. Ăn nhiều thực phẩm chứa cholesterol
C. Giảm cân, tập thể dục đều đặn và duy trì chế độ ăn uống lành mạnh
D. Hút thuốc lá"""

llm(
    f"""Translate the following string into English:
{query}

English Translation:
""",
)

In [ ]:
llm.create_chat_completion(
)

In [2]:
from langchain.llms import LlamaCpp
from llama_cpp.llama import Llama, LlamaGrammar
# import httpx
# grammar_text = httpx.get("https://raw.githubusercontent.com/ggerganov/llama.cpp/master/grammars/json_arr.gbnf").text
grammar = LlamaGrammar.from_file("../grammar/true_false.gbnf")

root ::= ws ANSWER ws 
ws ::= ws_4 
ANSWER ::= boolean | [T] [r] [u] [e] | [F] [a] [l] [s] [e] | [T] [R] [U] [E] | [F] [A] [L] [S] [E] 
boolean ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] 
ws_4 ::= [ <U+0009><U+000A>] ws_4 | 


from_string grammar:



In [6]:
prompt_dict = {
    2: """You are a smart medical question-answering that answer the question based on the provided context.
Use the following pieces of context to answer the question at the end. 
You must answer in the following json format with value true for the key with correct answer, otherwise false for the key with wrong answer:
{
    "A": ...,
    "B": ...
}

### Context:
{context}

### Question: {question}

### Answer:
""",
    3: """You are a smart medical question-answering that answer the question based on the provided context.
Use the following pieces of context to answer the question at the end. 
You must answer in the following json format with value true for the key with correct answer, otherwise false for the key with wrong answer:
{
    "A": ...,
    "B": ...,
    "C": ...
}

### Context:
{context}

### Question: {question}

### Answer:
""",
    4: """You are a smart medical question-answering that answer the question based on the provided context.
Use the following pieces of context to answer the question at the end.
You must answer in the following json format with value true for the key with correct answer, otherwise false for the key with wrong answer:
{
    "A": ...,
    "B": ...,
    "C": ...,
    "D": ...
}

### Context:
{context}

### Question: {question}

### Answer:
""",
    5: """You are a smart medical question-answering that answer the question based on the provided context.
Use the following pieces of context to answer the question at the end.
You must answer in the following json format with value true for the key with correct answer, otherwise false for the key with wrong answer:
{
    "A": ...,
    "B": ...,
    "C": ...,
    "D": ....
    "E": ...
}

### Context:
{context}

### Question: {question}

### Answer:
""",
    6: """You are a smart medical question-answering that answer the question based on the provided context.
Use the following pieces of context to answer the question at the end.
You must answer in the following json format with value true for the key with correct answer, otherwise false for the key with wrong answer:
{
    "A": ...,
    "B": ...,
    "C": ...,
    "D": ...,
    "E": ...,
    "F": ...
}

### Context:
{context}

### Question: {question}

### Answer:
""",
}
import json
with open("../prompts.json", "w", encoding="utf-8") as f:
    json.dump(prompt_dict, f, ensure_ascii=False, indent=4)

In [13]:
import pandas as pd
import numpy as np

test_df = pd.read_csv("/home/minhnam/Desktop/code/vietnamese-medical-qa/datasets/KALAPA_ByteBattles_2023_MEDICAL_Set1/public_test.csv")
test_df.head(10)


,id,question,option_1,option_2,option_3,option_4,option_5,option_6
0,level3_1,Hương đang mang thai và lo lắng mình có thể gặ...,A. Tuần 10,B.Tuần 20,C. Tuần 30,D. Tuần 40,NaN,NaN
1,level3_2,Hương đang mang thai tuần thứ 5 và lo lắng mìn...,A. 5 tuần,B. 15 tuần,C. 25 tuần,D. 35 tuần,NaN,NaN
2,level3_5,Có bao nhiêu loại rau tiền đạo biết rằng trong...,A. 2,B.3,C. 4,D. 5,NaN,NaN
3,level3_13,Bệnh nhân Dũng được chuẩn đoán bị viêm gan kéo...,Có,Không,NaN,NaN,NaN,NaN
4,level3_14,Một bệnh nhân bị đau tinh hoàn. Sau khi được h...,A. Nhiễm trùng đường tiết niệu,B.Ung thư tinh hoàn,C. Chấn thương,D. Giãn tĩnh mạch thừng tinh,NaN,NaN
5,level3_17,"An bị viêm cột sống, còn Nam bị tiêu chảy. Hỏi...",A. Bệnh lý tiêu hóa,B.Bệnh lý hô hấp,C. Bệnh lý tim mạch,D. Bệnh lý thần kinh,NaN,NaN
6,level3_22,Việc sử dụng thuốc kháng sinh một cách tự tiện...,A. Tăng cường hấp thu dinh dưỡng,B.Mất cân bằng hệ vi sinh đường ruột,C. Giảm nhu động ruột,D. Không có đáp án nào đúng,NaN,NaN
7,level3_27,Đàn ông có thể bị mắc ung thư vú không?,Có,Không,NaN,NaN,NaN,NaN
8,level3_35,Ung thư buồng trứng là khối u ác tính có xuất ...,A. Khám lâm sàng,B.Siêu âm qua ngả âm đạo và siêu âm ổ bụng,"C. Chẩn đoán hình ảnh, thăm dò chức năng và y ...",NaN,NaN,NaN
9,level3_42,"Một người khỏe mạnh ăn mào gà, không bị xuất h...",Có,Không,NaN,NaN,NaN,NaN


In [26]:
def preprocess_sample(sample):
    option_a = sample["option_1"]
    option_b = sample["option_2"]
    option_c = sample["option_3"]
    option_d = sample["option_4"]
    option_e = sample["option_5"]
    option_f = sample["option_6"]
    options = [option_a, option_b, option_c, option_d, option_e, option_f]

    new_samples = { "question": sample["question"] }
    num_available_options = 0
    for i, option in enumerate(options):
        if not pd.isnull(option):
            num_available_options += 1
            # options[i] = f"{chr(ord('A') + i)}. {option}"
            if options[i].startswith(("A", "B", "C", "D", "E", "F")):
                # remove the leading option letter
                options[i] = options[i][2:].strip()
            
            new_samples[f"{chr(ord('A') + i)}"] = options[i]
        else:
            break

    return num_available_options, new_samples

for i, row in test_df.iterrows():
    data = row.to_dict()
    print(json.dumps(data, indent=4, ensure_ascii=False))
    num_available_options, new_samples = preprocess_sample(data)
    print(json.dumps(new_samples, indent=4, ensure_ascii=False))
    print(num_available_options)
    break

{
    "id": "level3_1",
    "question": "Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?",
    "option_1": "A. Tuần 10",
    "option_2": "B.Tuần 20",
    "option_3": "C. Tuần 30",
    "option_4": "D. Tuần 40",
    "option_5": NaN,
    "option_6": NaN
}
{
    "question": "Hương đang mang thai và lo lắng mình có thể gặp phải rau tiền đạo. Hương có thể kiểm tra phát hiện bệnh này từ tuần thứ mấy của thai kỳ?",
    "A": "Tuần 10",
    "B": "Tuần 20",
    "C": "Tuần 30",
    "D": "Tuần 40"
}
4


In [23]:
json_data = [{'id': 'level3_1', 'answer': '0101'},
 {'id': 'level3_2', 'answer': '0110'},
 {'id': 'level3_5', 'answer': '0100'}]

json_df = pd.DataFrame(json_data)
json_df.to_csv("../test.csv", index=False)

In [25]:
# add tqdm for iterrows
for i, row in tqdm(json_df.iterrows(), total=len(json_df)):
    print(row["id"], row["answer"])